<h1 align = 'center'>Text Classification</h1>

<h3>Avinash Pasupulate</h3>
<br>
<a href='mailto:avinash.pasupulate@gmail.com'><i>avinash.pasupulate@gmail.com</i></a>

In [52]:
import os
import re
import copy
import string
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier


stopwrd = set(stopwords.words('english'))
cwd = os.getcwd()

Listing all files in directory

In [2]:
os.listdir(cwd+r'/data/reuters21578')

['feldman-cia-worldfactbook-data.txt',
 'all-orgs-strings.lc.txt',
 'reut2-004.sgm',
 'reut2-010.sgm',
 'all-exchanges-strings.lc.txt',
 'reut2-011.sgm',
 'reut2-005.sgm',
 'reut2-013.sgm',
 'reut2-007.sgm',
 'reut2-006.sgm',
 'reut2-012.sgm',
 'reut2-016.sgm',
 'reut2-002.sgm',
 'all-topics-strings.lc.txt',
 'reut2-003.sgm',
 'reut2-017.sgm',
 'reut2-001.sgm',
 'reut2-015.sgm',
 'reut2-014.sgm',
 'reut2-000.sgm',
 'reut2-019.sgm',
 'reut2-018.sgm',
 'reut2-020.sgm',
 'reut2-008.sgm',
 'reut2-009.sgm',
 'reut2-021.sgm',
 'README.txt',
 'all-people-strings.lc.txt',
 'cat-descriptions_120396.txt',
 'lewis.dtd',
 'all-places-strings.lc.txt']

In [3]:
data = cwd+r'/data/reuters21578'

In [4]:
#creating a list of filenames to import

l = list(range(0,22))
m = []
for i in l:
    if len(str(i))==1:
        filename = '/reut2-00'+str(i)+'.sgm'
        m.append(filename)
    else:
        filename = '/reut2-0'+str(i)+'.sgm'
        m.append(filename)

In [5]:
#create dataframe with extracted text

doc = pd.DataFrame()
for i in m:
    with open(data+'/reut2-002.sgm', 'r') as f:
        text = BeautifulSoup(f.read(), 'html.parser')
        documents = text.findAll('reuters')
        l=[]
        for a in documents:
            try:
                topic = a.topics.find_all('d')[0].contents[0]
                place = a.places.find_all('d')[0].contents[0]
                title = a.title.contents[0].lower()
                body = re.sub('\n', ' ', a.body.contents[0].lower())
                l.append([topic, place, title, body])
            except:
                pass
        doc = doc.append(l, ignore_index = True, sort = True)
doc.columns = ['topic', 'place', 'title', 'body']

In [6]:
#doc

In [7]:
print("unique categories in text: {:} in {:} rows"
      .format(len(doc['topic'].unique()),len(doc['topic'])))

unique categories in text: 36 in 11220 rows


In [8]:
lemmatizer = WordNetLemmatizer()

def remnum(x):
    return re.sub('[0-9]', '', x)

def rempunc(x):
    return ''.join([w for w in x if w not in string.punctuation])

def cltxt(x):
    tok = [rempunc(remnum(i)) for i in x.split() 
     if (i not in stopwrd)]
    cltxt1 = [lemmatizer.lemmatize(i) for i in tok if len(i)>2]
    return ' '.join(i for i in cltxt1)

In [9]:
doc['body'] = doc['body'].apply(cltxt)
doc['title'] = doc['title'].apply(cltxt)

In [10]:
doc

topic         place  \
0              earn            uk   
1             grain           usa   
2              earn           usa   
3            copper         chile   
4             crude        canada   
5             grain           usa   
6               acq        france   
7              ship        turkey   
8               acq           usa   
9              earn        canada   
10             earn        canada   
11         money-fx            uk   
12              acq           usa   
13         money-fx           usa   
14             earn           usa   
15              acq           usa   
16             earn           usa   
17             earn        canada   
18             earn           usa   
19              acq           usa   
20             earn           usa   
21            grain        france   
22            crude           usa   
23             earn           usa   
24             earn           usa   
25             earn           usa   
26              acq  west-germany   
27             earn           usa   
28             earn           usa   
29             earn            uk   
...             ...           ...   
11190         crude       ecuador   
11191          ship            uk   
11192          ship       belgium   
11193          earn      malaysia   
11194          earn      malaysia   
11195          earn      malaysia   
11196          earn      malaysia   
11197         sugar    bangladesh   
11198           gnp  west-germany   
11199      interest       bahrain   
11200          ship            uk   
11201         crude        turkey   
11202          ship        brazil   
11203         crude       ecuador   
11204       tapioca      thailand   
11205        cotton         china   
11206         sugar          ussr   
11207        rubber      thailand   
11208       carcass     australia   
11209      interest  west-germany   
11210    iron-steel         japan   
11211          gold     australia   
11212         trade    bangladesh   
11213           ipi   switzerland   
11214      money-fx     singapore   
11215  money-supply     singapore   
11216           ipi         china   
11217      money-fx   netherlands   
11218         crude         japan   
11219           acq            uk   

                                                title  \
0             jaguar see strong growth new model sale   
1              ccc accepts bonus bid wheat flour iraq   
2                      nord resource corp nrd qtr net   
3            quake damage chuquicamata mine spokesman   
4                orbit increase oil gas reserve value   
5                ccc accepts bonus bid semolina egypt   
6         five group apply buy french telephone group   
7                            blizzard close bosphorus   
8           japan fund jpn seeker confident financing   
9                  pegasus gold inc pgulf qtr dec net   
10                                keltic inc year net   
11           money market given help offered facility   
12                 hutton efh start purolator pcc bid   
13           yeutter put currency burden taiwan korea   
14        princeville development corp pvdc year loss   
15                        orangeco holder raise stake   
16                           horizon corp hzn qtr net   
17        international thomson organisation ltd year   
18             equatorial communication equa qtr loss   
19                     trimedyne tmed spin stake unit   
20              caseys general store inc casy qtr jan   
21      french free market cereal export bid detailed   
22       diamond shamrock dia raise crude oil posting   
23              oak hill sportswear corp ohsc qtr net   
24                      oak industry inc oak qtr loss   
25               regis corp rgis regular dividend set   
26                         mannesmann seek stake firm   
27                american intl group inc aig qtr net   
28                    system health care onefor split   
29       

In [11]:
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
encoder = preprocessing.LabelEncoder()
tfidf_vector = TfidfVectorizer(analyzer = 'word',
                               token_pattern = r'\w{1,}',
                               max_features = 5000, 
                              lowercase = False)

In [12]:
doc['label'] = encoder.fit_transform(doc['topic'])

In [18]:
from sklearn.model_selection import train_test_split

In [70]:
X_train, X_test, y_train, y_test = train_test_split(
    tfidf_vect, doc['label'], test_size = 0.25,
    random_state = 28)
actual = y_test

In [77]:
#change variable type and pass
tfidf_vector = tfidf_vector.fit(doc['body'])
X_train_tfidf = tfidf_vector.transform(X_train)
X_test_tfidf = tfidf_vector.transform(X_test)

TypeError: cannot use a string pattern on a bytes-like object

In [63]:
tfidf_fit

<11220x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 539418 stored elements in Compressed Sparse Row format>

In [ ]:
svcmod = SVC(kernel = 'rbf', C=0.1, gamma = 'auto')
svcfit = svcmod.fit(X_train, y_train)
svc_pred = svcfit.predict(X_test)
accuracy_score(actual, svc_pred)

In [47]:
mnb_mod = MultinomialNB()
mnb_fit = mnb_mod.fit(X_train, y_train)

In [48]:
mnb_score = accuracy_score(actual, mnb_fit.predict(X_test))

In [49]:
mnb_score

0.9818181818181818

In [56]:
from sklearn.linear_model import LogisticRegression

In [61]:
logmod = LogisticRegression()
logfit = logmod.fit(X_train, y_train)
accuracy_score(actual, logfit.predict(X_test))

/Users/Avinash/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Avinash/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


1.0

In [60]:
logfit.predict(X_test)

array([ 8, 22,  8, ...,  0,  8,  0])

In [55]:
rfmod = RandomForestClassifier()
rffit = rfmod.fit(X_train, y_train)
rf_score = accuracy_score(actual, rffit.predict(X_test))
rf_score

/Users/Avinash/anaconda3/lib/python3.5/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


1.0

In [31]:
mnb_fit.predict(X_test)

array([ 8, 22,  8, ...,  0,  8,  0])